# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import CSV file previously used
weather_data = "weather_data_df.csv"
weather_data_df = pd.read_csv(weather_data)
weather_data_df.head()

,Unnamed: 0,cities,lat,long,max_fahrenheit,humidity,cloudiness,wind_speed,country,date
0,0,Cabedelo,-6.98,-34.83,75.20,100,75,8.05,BR,1601259379
1,1,Lata,40.16,-8.33,64.00,87,0,2.17,PT,1601259380
2,2,Harlow,51.78,0.11,52.00,87,90,9.17,GB,1601259380
3,3,Hobart,-42.88,147.33,57.99,50,75,12.75,AU,1601259369
4,4,Belaya Gora,68.53,146.42,46.40,56,100,3.13,RU,1601259381


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
city_coord = weather_data_df[["lat","long"]]

In [4]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout = figure_layout)
markers = gmaps.marker_layer(city_coord)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
weather_data_df = weather_data_df.loc[weather_data_df['max_fahrenheit'] >70]
weather_data_df = weather_data_df.loc[weather_data_df['max_fahrenheit'] <80]

weather_data_df = weather_data_df.loc[weather_data_df['wind_speed'] <10]
weather_data_df = weather_data_df.loc[weather_data_df['cloudiness'] ==0]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_data_df.dropna()

hotel_coord = zip(hotel_df["lat"],hotel_df["long"])
radius = 5000
type_ = 'lodging'

hotels = []

#for hotel_coord in hotel_coord:

params = {
    'location': '33.87, 117.92', #(f'{hotel_coord[0]},{hotel_coord[1]}'),
    'radius': radius,
    'type': type_,
    'key':g_key}

        #print(params)

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(url, params=params)
hotel_data = response.json()
hotel_data
#except IndexError:
#print("error")
#hotels.append(      


{'html_attributions': [],
 'next_page_token': 'CqQCHAEAAHhNUU5R35mEpuSQZazfpwgjkmr7VfxLGhQZW1uT_1VQ0rKsBSZPgyI10sUPiYEPjMutPa0_twZDO_kePuSRIbvHAihOONUWC_u-q3gdK3S3qnhJB824PHycTMC_BE_2R6BdPFNv9EkzCV1ZeHlOrX64Q7e2T8FZQVi-5Lv3Z-GykGEbeDE9M5eb86eyEd5jci3O-KiGY1ZbvzkjIs9_pn2f-jPVssSVdxLTAokl1h6G1iKAykeukOQArDXFfI4nZF6hekSEM4MCw4SnhEK86M6raCIhebc6pGJrPmpntV8tGt96uc2kc30dwppHZVxCqbjDv5DAGdquBgF9NslElZaLOMsBh4zGES9vhgtsDZ-fJuqUnaulhNuebVFiLyeheBIQArTMKqR4SAbKlQ3pYiHKZxoUSLqxXwyrJA7hMCkaiTvKI3tkErk',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 33.892076, 'lng': 117.946319},
    'viewport': {'northeast': {'lat': 33.8934162302915,
      'lng': 117.9475496802915},
     'southwest': {'lat': 33.8907182697085, 'lng': 117.9448517197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'name': 'Elan Hotel Xuzhou HuiNing Pedestrian Street',
   'place_id': 'ChIJHRYdpTV-yDURJzPp5utG9r8',
   'plus_code': {'compound_code': 'VWRW+RG 

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
